In [1]:
from Bio import SeqIO
from tqdm import tqdm, tqdm_notebook
import numpy as np

from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.feature_extraction.text import CountVectorizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random_state = 4
device

device(type='cuda')

In [2]:
sg_reads_seq = []
sg_reads_tax = []

for seq_record in SeqIO.parse('data/16S-SG-reads.fa/16S-reads.fa', "fasta"):
    sg_reads_seq.append(seq_record.seq)
    sg_reads_tax.append(seq_record.description[seq_record.description.find('description=')+12+1:-1].split())
    
len(sg_reads_seq), len(sg_reads_tax)

(28224, 28224)

In [3]:
type(sg_reads_seq[0])

Bio.Seq.Seq

In [4]:
print(sg_reads_seq[0])

GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTTCTTGAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGAAGAGATCTGAAGGATTACCAGTGGCGAAGGCGGCCTACCGGCAACTATCCGACTCTCGGATGCGAAAGCGAGGTACCGGACGGGATCCGTTGAGCTGTATTTGCCCGCGGTAAATGGATTCCGCTAGCGGGACTCGGAGAGCCTTAGTGGAACCAC


In [5]:
print(sg_reads_tax[0])

['Gammaproteobacteria', 'Alteromonadales', 'Moritellaceae', 'Moritella']


In [6]:
tax = ["Class", "Order", "Family", "Genus"]

In [7]:
sg = pd.DataFrame()
for i in range(4):
    col = []
    for j in sg_reads_tax:
        if (len(j))==3:
            j.append('Simiduia')
        col.append(j[i])
    sg[tax[i]] = col

In [8]:
sg.head()

,Class,Order,Family,Genus
0,Gammaproteobacteria,Alteromonadales,Moritellaceae,Moritella
1,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Sagittula
2,Gammaproteobacteria,"\""Vibrionales\""",Vibrionaceae,Lucibacterium
3,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobaca
4,Gammaproteobacteria,Alteromonadales,Ferrimonadaceae,Ferrimonas


# Word Vectors using  Bag of Words model

In [9]:
human = pd.DataFrame()
human['sequence'] = [str(i) for i in sg_reads_seq]

In [10]:
human.head()

,sequence
0,GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGA...
1,ACTGCCTCCAAAACTATCAGTCTAGAGTTCGAGAGAGGTGAGTGGA...
2,TAGCGGCGTCGTTTGACGTTAGCGACAGAAGAAGCACCGGCTAACT...
3,CTAGCGTTGTTCGGAATTACTGGGCGTATAGCGCGCGTTAGGCGGA...
4,CATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTC...


In [11]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [12]:
human['words'] = human.apply(lambda x: getKmers(x['sequence'], size=11), axis=1) # default size = 6
human = human.drop('sequence', axis=1)

In [13]:
human.head()

,words
0,"[gccccgggctc, ccccgggctca, cccgggctcaa, ccgggc..."
1,"[actgcctccaa, ctgcctccaaa, tgcctccaaaa, gcctcc..."
2,"[tagcggcgtcg, agcggcgtcgt, gcggcgtcgtt, cggcgt..."
3,"[ctagcgttgtt, tagcgttgttc, agcgttgttcg, gcgttg..."
4,"[catggctcaga, atggctcagat, tggctcagatt, ggctca..."


In [14]:
human_texts = list(human['words'])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
print(human_texts[0])

gccccgggctc ccccgggctca cccgggctcaa ccgggctcaac cgggctcaacc gggctcaacct ggctcaacctg gctcaacctgg ctcaacctggg tcaacctggga caacctgggaa aacctgggaac acctgggaact cctgggaactg ctgggaactgc tgggaactgca gggaactgcat ggaactgcatt gaactgcattt aactgcatttc actgcatttcg ctgcatttcga tgcatttcgaa gcatttcgaac catttcgaact atttcgaactg tttcgaactgg ttcgaactggc tcgaactggca cgaactggcaa gaactggcaaa aactggcaaac actggcaaact ctggcaaacta tggcaaactag ggcaaactaga gcaaactagag caaactagagt aaactagagtt aactagagttc actagagttct ctagagttctt tagagttcttg agagttcttga gagttcttgag agttcttgaga gttcttgagag ttcttgagagg tcttgagaggg cttgagagggt ttgagagggtg tgagagggtgg gagagggtggt agagggtggta gagggtggtag agggtggtaga gggtggtagaa ggtggtagaat gtggtagaatt tggtagaattt ggtagaatttc gtagaatttca tagaatttcag agaatttcagg gaatttcaggt aatttcaggtg atttcaggtgt tttcaggtgta ttcaggtgtag tcaggtgtagc caggtgtagcg aggtgtagcgg ggtgtagcggt gtgtagcggtg tgtagcggtga gtagcggtgaa tagcggtgaaa agcggtgaaat gcggtgaaatg cggtgaaatgc ggtgaaatgcg gtgaaatgcga tgaaatgcgaa gaaa

In [15]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,4)) # initial : (4, 4)
X = cv.fit_transform(human_texts)

In [17]:
X.shape

(28224, 11386484)

In [16]:
cv.vocabulary_

{'gccccgggctc': 6635508,
 'ccccgggctca': 3749443,
 'cccgggctcaa': 3790631,
 'ccgggctcaac': 3961284,
 'cgggctcaacc': 4667272,
 'gggctcaacct': 7703298,
 'ggctcaacctg': 7551955,
 'gctcaacctgg': 7017458,
 'ctcaacctggg': 5084694,
 'tcaacctggga': 9441159,
 'caacctgggaa': 2925693,
 'aacctgggaac': 258618,
 'acctgggaact': 1055488,
 'cctgggaactg': 4120235,
 'ctgggaactgc': 5346011,
 'tgggaactgca': 10510079,
 'gggaactgcat': 7597327,
 'ggaactgcatt': 7173812,
 'gaactgcattt': 5662227,
 'aactgcatttc': 339480,
 'actgcatttcg': 1351922,
 'ctgcatttcga': 5286499,
 'tgcatttcgaa': 10269441,
 'gcatttcgaac': 6570166,
 'catttcgaact': 3528042,
 'atttcgaactg': 2843188,
 'tttcgaactgg': 11306176,
 'ttcgaactggc': 11011021,
 'tcgaactggca': 9726665,
 'cgaactggcaa': 4204631,
 'gaactggcaaa': 5663721,
 'aactggcaaac': 343770,
 'actggcaaact': 1366018,
 'ctggcaaacta': 5333336,
 'tggcaaactag': 10456987,
 'ggcaaactaga': 7380517,
 'gcaaactagag': 6379885,
 'caaactagagt': 2881861,
 'aaactagagtt': 75536,
 'aactagagttc': 316515,
 

# Class 

In [18]:
print(list(sg.groupby('Class').count().index), len(sg.groupby('Class').count().index))

['Alphaproteobacteria', 'Betaproteobacteria', 'Gammaproteobacteria'] 3


In [19]:
classes = list(pd.get_dummies(sg['Class']).columns)
classes = ['class_' + i for i in classes]
print(classes)

['class_Alphaproteobacteria', 'class_Betaproteobacteria', 'class_Gammaproteobacteria']


In [20]:
labels_classes = np.argmax(np.array(pd.get_dummies(sg['Class'])), axis=1)
print(labels_classes, len(labels_classes))

[2 0 2 ... 0 0 2] 28224


In [21]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, 
                                                    labels_classes, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [22]:
print(X_train_c.shape)
print(X_test_c.shape)

(25401, 11386484)
(2823, 11386484)


In [23]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_c = MultinomialNB(alpha=0.1)
classifier_c.fit(X_train_c, y_train_c)

MultinomialNB(alpha=0.1)

In [24]:
y_pred_c = classifier_c.predict(X_test_c)
y_pred_c

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_c, name='Actual'), pd.Series(y_pred_c, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_c, y_pred_c)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted     0    1     2
Actual                    
0          1281   22     0
1             0  195     0
2             1   33  1291
accuracy = 0.980 
precision = 0.984 
recall = 0.980 
f1 = 0.981


In [26]:
def predict_c(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_c.predict(cv.transform(seq))
def find_class(seq):
    pred = predict_c(seq)
    return [classes[i] for i in pred]

In [27]:
predict_c(human_texts[0]).item()

2

In [28]:
find_class(human_texts[0:5]), find_class(human_texts[0])

(['class_Gammaproteobacteria',
  'class_Alphaproteobacteria',
  'class_Gammaproteobacteria',
  'class_Alphaproteobacteria',
  'class_Gammaproteobacteria'],
 ['class_Gammaproteobacteria'])

# Order

In [29]:
print(list(sg.groupby('Order').count().index), len(sg.groupby('Order').count().index))

['Aeromonadales', 'Alteromonadales', 'Caulobacterales', 'Chromatiales', 'Ferrovales', 'Gammaproteobacteria_incertae_sedis', 'Legionellales', 'Methylophilales', 'Neisseriales', 'Oceanospirillales', 'Pasteurellales', 'Pseudomonadales', 'Rhizobiales', 'Rhodobacterales', 'Rhodospirillales', 'Sphingomonadales', 'Thiotrichales', 'Vibrionales', 'Xanthomonadales', '\\"Salinisphaerales\\"', '\\"Vibrionales\\"'] 21


In [30]:
orders = []
for i in sg['Order']:
    if i[0:2]==   r"\"":
        i = i[2:-2]
    orders.append(i)
sg['Order'] = orders

In [32]:
print(list(sg.groupby('Order').count().index), len(sg.groupby('Order').count().index))

['Aeromonadales', 'Alteromonadales', 'Caulobacterales', 'Chromatiales', 'Ferrovales', 'Gammaproteobacteria_incertae_sedis', 'Legionellales', 'Methylophilales', 'Neisseriales', 'Oceanospirillales', 'Pasteurellales', 'Pseudomonadales', 'Rhizobiales', 'Rhodobacterales', 'Rhodospirillales', 'Salinisphaerales', 'Sphingomonadales', 'Thiotrichales', 'Vibrionales', 'Xanthomonadales'] 20


In [33]:
orders = list(pd.get_dummies(sg['Order']).columns)
orders = ['order_' + i for i in orders]

labels_orders = np.argmax(np.array(pd.get_dummies(sg['Order'])), axis=1)
print(labels_orders, len(labels_orders))

[ 1 13 18 ... 13 12 19] 28224


In [34]:
print(orders)

['order_Aeromonadales', 'order_Alteromonadales', 'order_Caulobacterales', 'order_Chromatiales', 'order_Ferrovales', 'order_Gammaproteobacteria_incertae_sedis', 'order_Legionellales', 'order_Methylophilales', 'order_Neisseriales', 'order_Oceanospirillales', 'order_Pasteurellales', 'order_Pseudomonadales', 'order_Rhizobiales', 'order_Rhodobacterales', 'order_Rhodospirillales', 'order_Salinisphaerales', 'order_Sphingomonadales', 'order_Thiotrichales', 'order_Vibrionales', 'order_Xanthomonadales']


In [35]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, 
                                                    labels_orders, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [36]:
print(X_train_o.shape)
print(X_test_o.shape)

(25401, 11386484)
(2823, 11386484)


In [37]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_o = MultinomialNB(alpha=0.1)
classifier_o.fit(X_train_o, y_train_o)

MultinomialNB(alpha=0.1)

In [38]:
y_pred_o = classifier_o.predict(X_test_o)
y_pred_o

array([19, 13, 13, ..., 13,  5, 13], dtype=int64)

In [39]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_o, name='Actual'), pd.Series(y_pred_o, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_o, y_pred_o)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0    1    2   3   4   5   6   7    8    9    10  11   12   13  14  \
Actual                                                                         
0          20    0    0   0   0   0   0   0    0    0    0   0    0    0   0   
1           1  309    0   0   0   4   0   0    0    8    0   4    0    0   0   
2           0    0  114   0   0   0   0   0    0    0    0   0    0    0   0   
3           0    0    0  48   0   0   0   0    0    0    0   0    0    0   0   
4           0    0    0   0  37   0   0   0    0    0    0   0    0    0   0   
5           0    5    0   0   0  21   0   0    0    1    0   0    0    0   0   
6           0    0    0   0   0   0  57   0    0    0    0   0    0    0   0   
7           0    0    0   0   0   0   0  19    1    0    0   0    0    0   0   
8           0    0    0   0   0   0   0   0  138    0    0   0    0    0   0   
9           0    4    0   0   1   2   1   0    0  310    0   1    0    0   0   
10          0    0    

In [40]:
def predict_o(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_o.predict(cv.transform(seq))

def find_order(seq):
    pred = predict_o(seq)
    return [orders[i] for i in pred]

In [41]:
predict_o(human_texts[3]).item()

13

In [42]:
find_order(human_texts[0:5])

['order_Alteromonadales',
 'order_Rhodobacterales',
 'order_Vibrionales',
 'order_Rhodobacterales',
 'order_Alteromonadales']

# Family

In [43]:
print(list(sg.groupby('Family').count().index),'\n', len(sg.groupby('Family').count().index))

['Aeromonadaceae', 'Alteromonadaceae', 'Alteromonadales_incertae_sedis', 'Bradyrhizobiaceae', 'Brucellaceae', 'Caulobacteraceae', 'Chromatiaceae', 'Colwelliaceae', 'Coxiellaceae', 'Erythrobacteraceae', 'Ferrimonadaceae', 'Ferrovaceae', 'Hahellaceae', 'Halomonadaceae', 'Hyphomicrobiaceae', 'Hyphomonadaceae', 'Idiomarinaceae', 'Legionellaceae', 'Methylophilaceae', 'Moritellaceae', 'Neisseriaceae', 'Oceanospirillaceae', 'Oceanospirillales_incertae_sedis', 'Pasteurellaceae', 'Phyllobacteriaceae', 'Piscirickettsiaceae', 'Pseudoalteromonadaceae', 'Pseudomonadaceae', 'Psychromonadaceae', 'Rhizobiaceae', 'Rhodobacteraceae', 'Rhodospirillaceae', 'Simiduia', 'Sinobacteraceae', 'Sphingomonadaceae', 'Vibrionaceae', 'Xanthobacteraceae', 'Xanthomonadaceae', '\\"Salinisphaeraceae\\"'] 
 39


In [44]:
families = []
for i in sg['Family']:
    if i[0:2]==   r"\"":
        i = i[2:-2]
    families.append(i)
    
sg['Family'] = families

In [46]:
print(list(sg.groupby('Family').count().index), len(sg.groupby('Family').count().index))

['Aeromonadaceae', 'Alteromonadaceae', 'Alteromonadales_incertae_sedis', 'Bradyrhizobiaceae', 'Brucellaceae', 'Caulobacteraceae', 'Chromatiaceae', 'Colwelliaceae', 'Coxiellaceae', 'Erythrobacteraceae', 'Ferrimonadaceae', 'Ferrovaceae', 'Hahellaceae', 'Halomonadaceae', 'Hyphomicrobiaceae', 'Hyphomonadaceae', 'Idiomarinaceae', 'Legionellaceae', 'Methylophilaceae', 'Moritellaceae', 'Neisseriaceae', 'Oceanospirillaceae', 'Oceanospirillales_incertae_sedis', 'Pasteurellaceae', 'Phyllobacteriaceae', 'Piscirickettsiaceae', 'Pseudoalteromonadaceae', 'Pseudomonadaceae', 'Psychromonadaceae', 'Rhizobiaceae', 'Rhodobacteraceae', 'Rhodospirillaceae', 'Salinisphaeraceae', 'Simiduia', 'Sinobacteraceae', 'Sphingomonadaceae', 'Vibrionaceae', 'Xanthobacteraceae', 'Xanthomonadaceae'] 39


In [47]:
families = list(pd.get_dummies(sg['Family']).columns)
families = ['familiy_' + i for i in families]

labels_families = np.argmax(np.array(pd.get_dummies(sg['Family'])), axis=1)
print(labels_families, len(labels_families))

[19 30 36 ... 30 37 38] 28224


In [48]:
print(families)

['familiy_Aeromonadaceae', 'familiy_Alteromonadaceae', 'familiy_Alteromonadales_incertae_sedis', 'familiy_Bradyrhizobiaceae', 'familiy_Brucellaceae', 'familiy_Caulobacteraceae', 'familiy_Chromatiaceae', 'familiy_Colwelliaceae', 'familiy_Coxiellaceae', 'familiy_Erythrobacteraceae', 'familiy_Ferrimonadaceae', 'familiy_Ferrovaceae', 'familiy_Hahellaceae', 'familiy_Halomonadaceae', 'familiy_Hyphomicrobiaceae', 'familiy_Hyphomonadaceae', 'familiy_Idiomarinaceae', 'familiy_Legionellaceae', 'familiy_Methylophilaceae', 'familiy_Moritellaceae', 'familiy_Neisseriaceae', 'familiy_Oceanospirillaceae', 'familiy_Oceanospirillales_incertae_sedis', 'familiy_Pasteurellaceae', 'familiy_Phyllobacteriaceae', 'familiy_Piscirickettsiaceae', 'familiy_Pseudoalteromonadaceae', 'familiy_Pseudomonadaceae', 'familiy_Psychromonadaceae', 'familiy_Rhizobiaceae', 'familiy_Rhodobacteraceae', 'familiy_Rhodospirillaceae', 'familiy_Salinisphaeraceae', 'familiy_Simiduia', 'familiy_Sinobacteraceae', 'familiy_Sphingomonadac

In [49]:
len(families)

39

In [50]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X, 
                                                    labels_families, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [51]:
print(X_train_f.shape)
print(X_test_f.shape)

(25401, 11386484)
(2823, 11386484)


In [52]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_f = MultinomialNB(alpha=0.1)
classifier_f.fit(X_train_f, y_train_f)

MultinomialNB(alpha=0.1)

In [53]:
y_pred_f = classifier_f.predict(X_test_f)
y_pred_f

array([38, 30, 30, ..., 30, 33, 30], dtype=int64)

In [54]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_f, name='Actual'), pd.Series(y_pred_f, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_f, y_pred_f)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0    1   2   3   4   5   6   7   8   9   ...  29   30  31  32  33  \
Actual                                              ...                        
0          19    0   0   0   0   0   0   0   0   0  ...   0    0   0   0   0   
1           0  115   0   0   0   0   0   0   0   0  ...   0    0   0   0   0   
2           0    0  29   0   0   0   0   0   0   0  ...   0    0   0   0   1   
3           0    0   0  76   0   0   0   0   0   0  ...   0    0   0   0   0   
4           0    0   0   0  28   0   0   0   0   0  ...   0    0   0   0   0   
5           0    0   0   0   0  60   0   0   0   0  ...   0    0   0   0   0   
6           0    0   0   0   0   0  48   0   0   0  ...   0    0   0   0   0   
7           0    1   0   0   0   0   0  27   0   0  ...   0    0   0   0   0   
8           0    0   0   0   0   0   1   0  28   0  ...   0    0   0   0   0   
9           0    0   0   0   0   0   0   0   0  47  ...   0    0   0   0   0   
10          0    0   0

In [55]:
def predict_f(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_f.predict(cv.transform(seq))

def find_family(seq):
    pred = predict_f(seq)
    return [families[i] for i in pred]

In [56]:
predict_f(human_texts[3]).item()

30

In [57]:
find_family(human_texts[0:5])

['familiy_Moritellaceae',
 'familiy_Rhodobacteraceae',
 'familiy_Vibrionaceae',
 'familiy_Rhodobacteraceae',
 'familiy_Ferrimonadaceae']

# GENUS

In [58]:
print(list(sg.groupby('Genus').count().index),'\n', len(sg.groupby('Genus').count().index))

['Actinobacillus', 'Afipia', 'Agarivorans', 'Aggregatibacter', 'Algimonas', 'Aliidiomarina', 'Aliiroseovarius', 'Allochromatium', 'Alteromonas', 'Amphritea', 'Amylibacter', 'Ancylobacter', 'Aquaspirillum', 'Azorhizobium', 'Brevundimonas', 'Caulobacter', 'Celeribacter', 'Cellvibrio', 'Chelonobacter', 'Chitinibacter', 'Chromohalobacter', 'Cobetia', 'Coxiella', 'Croceicoccus', 'Erythrobacter', 'Falsochrobactrum', 'Ferrimonas', 'Ferrovum', 'Hahella', 'Halovibrio', 'Henriciella', 'Histophilus', 'Kingella', 'Labrys', 'Legionella', 'Leisingera', 'Loktanella', 'Lucibacterium', 'Luteibacter', 'Marinobacter', 'Marinomonas', 'Marinospirillum', 'Maritalea', 'Marivita', 'Mesorhizobium', 'Methylobacillus', 'Moritella', 'Nautella', 'Necropsobacter', 'Neisseria', 'Neorhizobium', 'Nevskia', 'Nicoletella', 'Nitrincola', 'Nitrosococcus', 'Niveispirillum', 'Novispirillum', 'Oceanisphaera', 'Pannonibacter', 'Paraglaciecola', 'Pedomicrobium', 'Pelagibacterium', 'Pelagibius', 'Phaeobacter', 'Phocoenobacter',

In [59]:
genes = list(pd.get_dummies(sg['Genus']).columns)
genes = ['genus_' + i for i in genes]

labels_genes = np.argmax(np.array(pd.get_dummies(sg['Genus'])), axis=1)
print(labels_genes, len(labels_genes))

[46 82 37 ...  6 13 92] 28224


In [60]:
print(genes)
len(genes)

['genus_Actinobacillus', 'genus_Afipia', 'genus_Agarivorans', 'genus_Aggregatibacter', 'genus_Algimonas', 'genus_Aliidiomarina', 'genus_Aliiroseovarius', 'genus_Allochromatium', 'genus_Alteromonas', 'genus_Amphritea', 'genus_Amylibacter', 'genus_Ancylobacter', 'genus_Aquaspirillum', 'genus_Azorhizobium', 'genus_Brevundimonas', 'genus_Caulobacter', 'genus_Celeribacter', 'genus_Cellvibrio', 'genus_Chelonobacter', 'genus_Chitinibacter', 'genus_Chromohalobacter', 'genus_Cobetia', 'genus_Coxiella', 'genus_Croceicoccus', 'genus_Erythrobacter', 'genus_Falsochrobactrum', 'genus_Ferrimonas', 'genus_Ferrovum', 'genus_Hahella', 'genus_Halovibrio', 'genus_Henriciella', 'genus_Histophilus', 'genus_Kingella', 'genus_Labrys', 'genus_Legionella', 'genus_Leisingera', 'genus_Loktanella', 'genus_Lucibacterium', 'genus_Luteibacter', 'genus_Marinobacter', 'genus_Marinomonas', 'genus_Marinospirillum', 'genus_Maritalea', 'genus_Marivita', 'genus_Mesorhizobium', 'genus_Methylobacillus', 'genus_Moritella', 'ge

100

In [61]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X, 
                                                    labels_genes, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [62]:
print(X_train_g.shape)
print(X_test_g.shape)

(25401, 11386484)
(2823, 11386484)


In [63]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_g = MultinomialNB(alpha=0.1)
classifier_g.fit(X_train_g, y_train_g)

MultinomialNB(alpha=0.1)

In [64]:
y_pred_g = classifier_g.predict(X_test_g)
y_pred_g

array([96, 67, 10, ..., 47, 87, 69], dtype=int64)

In [65]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_g, name='Actual'), pd.Series(y_pred_g, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_g, y_pred_g)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0   1   2   3   4   5   6   7   8   9   ...  90  91  92  93  94  \
Actual                                             ...                       
0          17   0   0   1   0   0   0   0   0   0  ...   0   0   0   0   0   
1           0  16   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   
2           0   0  26   0   0   0   0   0   0   0  ...   0   0   0   0   0   
3           0   0   0  39   0   0   0   0   0   0  ...   0   0   0   0   0   
4           0   0   0   0  29   0   0   0   0   0  ...   0   0   0   0   0   
...        ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..   
95          0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   
96          0   0   0   0   0   0   0   0   0   0  ...   0   0   4   0   0   
97          0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   
98          0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   
99          0   0   0   0   0   0   0   0   0 

In [66]:
def predict_g(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_g.predict(cv.transform(seq))

def find_genus(seq):
    pred = predict_g(seq)
    return [genes[i] for i in pred]

In [67]:
predict_g(human_texts[3]).item()

77

In [68]:
find_genus(human_texts[0:5])

['genus_Moritella',
 'genus_Sagittula',
 'genus_Lucibacterium',
 'genus_Rhodobaca',
 'genus_Ferrimonas']

# Test

In [69]:
def predict(seq):
    if type(seq) != list:
        seq = [seq]

    t = pd.DataFrame()
    t['sequence'] = [str(i) for i in seq]
    t_texts = list(t.apply(lambda x: getKmers(x['sequence']), axis=1))
    for item in range(len(t_texts)):
        t_texts[item] = ' '.join(t_texts[item])
    
    class_ = find_class(t_texts)
    order = find_order(t_texts)
    family = find_family(t_texts)
    genus = find_genus(t_texts)
    
    t['Class'] = class_
    t['Order'] = order
    t['Family'] = family
    t['Genus'] = genus
    return t

In [70]:
predict(sg_reads_seq[0:5])

,sequence,Class,Order,Family,Genus
0,GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGA...,class_Alphaproteobacteria,order_Rhizobiales,familiy_Rhodobacteraceae,genus_Zymobacter
1,ACTGCCTCCAAAACTATCAGTCTAGAGTTCGAGAGAGGTGAGTGGA...,class_Alphaproteobacteria,order_Rhizobiales,familiy_Rhodobacteraceae,genus_Zymobacter
2,TAGCGGCGTCGTTTGACGTTAGCGACAGAAGAAGCACCGGCTAACT...,class_Alphaproteobacteria,order_Rhizobiales,familiy_Rhodobacteraceae,genus_Zymobacter
3,CTAGCGTTGTTCGGAATTACTGGGCGTATAGCGCGCGTTAGGCGGA...,class_Alphaproteobacteria,order_Rhizobiales,familiy_Rhodobacteraceae,genus_Zymobacter
4,CATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTC...,class_Alphaproteobacteria,order_Rhizobiales,familiy_Rhodobacteraceae,genus_Zymobacter


In [71]:
pd.DataFrame(sg_reads_tax[0:5])

,0,1,2,3
0,Gammaproteobacteria,Alteromonadales,Moritellaceae,Moritella
1,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Sagittula
2,Gammaproteobacteria,"\""Vibrionales\""",Vibrionaceae,Lucibacterium
3,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobaca
4,Gammaproteobacteria,Alteromonadales,Ferrimonadaceae,Ferrimonas


In [72]:
print(sg_reads_seq[0])

GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTTCTTGAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGAAGAGATCTGAAGGATTACCAGTGGCGAAGGCGGCCTACCGGCAACTATCCGACTCTCGGATGCGAAAGCGAGGTACCGGACGGGATCCGTTGAGCTGTATTTGCCCGCGGTAAATGGATTCCGCTAGCGGGACTCGGAGAGCCTTAGTGGAACCAC
